# Gradient Descent with libraries

# Using libraries

So now we've seen how linear regression works analytically. Although it does not seem to be too hard to program a regression algorithm, there can be a lot of complications. Things work differently when for instance the data set is \*big\* and when there are many features. This means that if we code everything from scratch, we have to be very careful not to make mistakes. Another issue is that Gradient Descent sometimes converges very slowly and although speeding up learning is possible, it also makes the program quite a bit more complex.

Alternatively, we can use an existing implementation. For Python, may popular algorithms have already been written an added to an open source library. These algorithms are often rigorously  tested, well documented and very efficient (in fact, most algorithms were implemented in a fast language like Fortran or C++ and made accessible from Python). Choosing an existing implementation makes coding \*a lot\* easier and you are far less likely to make mistakes. So why is Python most popular for scripting Data Science experiments and not one of these fast languages? This is probably partly because Python is comprehensive, compact, highly readable, interactive and very easy to learn and use. We have seen that a less efficient language does have its disadvantages for processing large-data volumes, but if the libraries we use do the heavy lifting we will not notice much drop in performance.

# Data

When loading the data for use with a library, it is important to read in what format we should provide it. Luckily, for Python, Numpy array's are like the standard format everyone uses, because it's speed, stability and versatilty is unrivaled. However, for some choices, like should we provide the data in row-vectors or column-vectors, should we add a bias or does the library handle that, you should check out an example to see how that works.

In our case, SKlearn automatically adds a bias, so we should not add a bias ourselves. The data loaders that we use will only add a bias if we add bias=True.

In [1]:
from ml import *
data = wines_quality_alcohol()

# Model

The **SKLearn** library (pronounce Sci-Kit Learn) contains excellent implementations for many Machine Learning algorithms. To use these libraries it is crucial that you read the documentation or some tutorial on how to use these properly. To use Gradient Descent to estimate a linear regression function, we should use the `partial_fit()` method on the `SGDRegressor`. We can configure the `loss` function to `squared_loss` and `eta0` is the learning rate.

In this experiment, we turn off penalty and shuffle to obtain results close to what we computed with our own code.

In [2]:
model = SGDRegressor(eta0=3e-4, learning_rate='invscaling', penalty = None, shuffle=False)

# Training

Next is the SKLearn equivalent that the ML library uses. Slightly advanced is that we may need a so-called adaptive strategy for the learning rate to converge smoothly. In this case invscaling means that the learning rate is gradually lowered. This helps to prevent overshooting of the optimum. The learning rate itself is called eta0. 

The SGDRegressor does provide a `fit` method. Unfortunately, SKLearn does not provide the loss, so we have to compute the loss ourselves in the loop. And when we check the loss we see that `fit` actually gives far from optimal results. This has to do with Gradient Descent stopping too early.

In [3]:
model.fit(data.train_X, data.train_y)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.0003, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty=None, power_t=0.25,
       random_state=None, shuffle=False, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [4]:
y_predict = model.predict(data.train_X)
mean_squared_error(y_predict, data.train_y)

0.5452359088674495

Instead, we can control learning by using partial_fit to update the model by training one epoch each time (one pass over the training set) until we are done. We should see that the loss goes down to about what we obtained with the Normal Equation.

#### In the partial_fit function, fill in the training data on the ... and train the model

In [6]:
for _ in range(100001):
    model.partial_fit( data.train_X, data.train_y )
    if _ % 10000 == 0:
        y_predict = model.predict(data.train_X)
        print(mean_squared_error(y_predict, data.train_y))

0.5452092901900766
0.5188985845933729
0.5149812015933966
0.5138676873897504
0.5134932306563138
0.5133546524847034
0.5132999245857277
0.5132772452919355
0.5132674872366543
0.5132631596802645
0.5132611921638199


Note that if we check the coefficients, they are slightly different. The most likely explanation is that there is a very wide local minimum in this case, perhaps caused by the discreteness of the target variable.

In [7]:
model.intercept_, model.coef_

(array([1.80872459]), array([0.36796593]))